In [ ]:
!pip install scprep phate magic-impute mnnpy scanpy loompy scikit-misc iMAP bbknn

In [ ]:
import mnnpy

In [ ]:
mnnpy.utils

In [ ]:
pip install git+https://github.com/theislab/scib.git

In [ ]:
#### IMPORT LIBRARY ####
import scanpy as sc
import scprep
import copy
from collections import Counter
import umap.umap_ as umap
import pandas as pd
import scIB

In [ ]:
def preprocessing(adata):
  sc.pp.filter_cells(adata,3)
  sc.pp.filter_genes(adata,200)
  sc.pp.normalize_per_cell(adata,1e4)
  sc.pp.log1p(adata)
  return adata

In [ ]:
import rpy2
%load_ext rpy2.ipython

In [ ]:
adata = sc.read_loom('/content/drive/MyDrive/data/cell_lines.loom',sparse=False)  #Load cell line dataset(-> count data). 

In [ ]:
adata_old = copy.deepcopy(adata)

# raw

In [ ]:
c=Counter(adata.obs['batch'])

In [ ]:
adata = preprocessing(adata)

In [ ]:
adata_now = sc.AnnData(adata.X)

In [ ]:
adata_now.obs['batch'] = adata.obs['batch']
adata_now.obs['celltype'] = adata.obs['celltype']
adata_now.obs_names = adata.obs_names

In [ ]:
adata_now.write_loom('cl_liger.loom')

# mnn

In [ ]:
import mnnpy

In [ ]:
mnnpy.utils

In [ ]:
c = Counter(adata_old.obs['batch'])

In [ ]:
c = dict(c)
c

In [ ]:
adata_old = imap.stage1.data_preprocess(adata_old)

In [ ]:
adata1 = adata_old[adata_old.obs['batch'] == '293t']
adata2 = adata_old[adata_old.obs['batch'] == 'Jurkat']
adata3 = adata_old[adata_old.obs['batch'] == 'Mix']

In [ ]:
cdata = mnnpy.mnn_correct(adata1, adata2, adata3, svd_dim=50,batch_key = 'batch', batch_categories=['293t', 'Jurkat', 'Mix'])[0]

In [ ]:
cdata

# BBKNN


In [ ]:
import bbknn

In [ ]:
sc.tl.pca(adata_old)
bbknn.bbknn(adata_old, batch_key='batch')

In [ ]:
sc.tl.umap(adata_old)

In [ ]:
adata_old.obs['batch'] = adata_old.obs['batch'].astype('category')
adata_old.obs['celltype'] = adata_old.obs['celltype'].astype('category')

In [ ]:
sc.pl.umap(adata_old, color=['celltype'])

# IMAP


In [ ]:
adata = imap.stage1.data_preprocess(adata)

In [ ]:
### Stage I
EC, ec_data = imap.stage1.iMAP_fast(adata, key="batch", n_epochs=200) 

### Stage II
output_results = imap.stage2.integrate_data(adata, ec_data, inc = False, n_epochs=300)

# Harmony

In [ ]:
!pip install harmonypy

In [ ]:
import scprep
import imap  #used for feature detected
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import phate
import graphtools as gt
import magic
import os
import datetime
import scanpy as sc
from skmisc.loess import loess
import sklearn.preprocessing as preprocessing
import harmonypy as harmony
import umap.umap_ as umap

In [ ]:
import torch.autograd
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
import torch.utils.data as Data  #Data是用来批训练的模块
from torchvision.utils import save_image
import numpy as np
import os
import pandas as pd
import torch.optim.lr_scheduler as lr_s 

In [ ]:
from collections import Counter
import copy
import loompy

In [ ]:
adata = sc.read_loom('/content/drive/MyDrive/data/cell_lines.loom',sparse=False)  #Load cell line dataset(-> count data). 
adata_old = copy.deepcopy(adata)

In [ ]:
adata_old = imap.stage1.data_preprocess(adata_old)

In [ ]:
metadata = pd.DataFrame(adata_old.obs['batch'], columns=['cellname', 'batch'])

In [ ]:
output_results = harmony.run_harmony(adata_old.X, metadata, vars_use=['batch'])

In [ ]:
result = output_results.Z_corr.T

In [ ]:
res = pd.DataFrame(result)

# scVI

In [ ]:
import sys

#if branch is stable, will install via pypi, else will install from source
branch = "stable"
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB and branch == "stable":
    !pip install --quiet scvi-tools[tutorials]
elif IN_COLAB and branch != "stable":
    !pip install --quiet --upgrade jsonschema
    !pip install --quiet git+https://github.com/yoseflab/scvi-tools@$branch#egg=scvi-tools[tutorials]

In [ ]:
import scanpy as sc
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scvi
import copy
import umap.umap_ as umap
%matplotlib inline

In [ ]:
adata = sc.read_loom('cell_lines.loom',sparse=False)  #Load cell line dataset(-> count data). 

In [ ]:
adata.layers['counts'] = adata.X.copy()
#Use raw count, accoridng the tutorial
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    flavor="seurat_v3",
    batch_key="batch",
    subset=True,
    layer="counts"
)


scvi.data.setup_anndata(adata, layer="counts")


model = scvi.model.SCVI(adata,n_latent=30)

model.train(max_epochs=200)

result = model.get_latent_representation()